In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Unzip a file stored in Google Drive
!unzip "/content/drive/MyDrive/Tokenised_Datasets/tokenized_datasets-20251116T165959Z-1-001.zip" -d "/content/drive/MyDrive/Unzip_Tokenised"


Archive:  /content/drive/MyDrive/Tokenised_Datasets/tokenized_datasets-20251116T165959Z-1-001.zip
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/label_to_id.pkl  
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/dataset_dict.json  
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/test/state.json  
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/validation/state.json  
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/validation/dataset_info.json  
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/test/data-00000-of-00001.arrow  
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/train/dataset_info.json  
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/test/dataset_info.json  
  inflating: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets/train/state.json  
  inflating: /content/drive/MyDrive/Unzip_Tokenis

In [ ]:
# ============================================================
# 0. Install dependencies (Colab)
# ============================================================
!pip install -q "transformers>=4.36.0" "datasets" "accelerate" "bitsandbytes" "peft" "scikit-learn"
!pip install -q "transformers>=4.36.0" "datasets" "accelerate" "bitsandbytes" "peft" "scikit-learn"


import transformers
print(transformers.__version__)  # make sure it's >= 4.36.0

from transformers import TrainingArguments

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 10.6 MB/s eta 0:00:00
4.57.1


In [ ]:
# 2. Imports & config
# ============================================================
import os
from typing import Dict, Any

import torch
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


from datasets import load_from_disk, DatasetDict

base_path = "/content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets"  # change to the real name if needed

train_ds = load_from_disk(f"{base_path}/train")
val_ds   = load_from_disk(f"{base_path}/validation")  # or "valid" if that's what you see
test_ds  = load_from_disk(f"{base_path}/test")

dataset = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds,
})
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 18391
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2299
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2299
    })
})


In [ ]:
!pip install -q "transformers>=4.36.0" "datasets" "accelerate" "bitsandbytes" "peft" "scikit-learn"

import transformers
print(transformers.__version__)

4.57.1


In [7]:

# 2. Imports & config
# ============================================================
import os
from typing import Dict, Any

import torch
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# ---- EDIT THESE PATHS ----
tokenized_dataset_path = "/content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets"  # <--- CHANGE
final_model_dir        = "/content/drive/MyDrive/finbert_qlora_finetuned"               # <--- CHANGE

base_model_name = "ProsusAI/finbert"  # FinBERT

# Training hyperparameters (adjust if needed)
num_train_epochs = 3
per_device_train_batch_size = 8
per_device_eval_batch_size = 16
learning_rate = 2e-4
weight_decay = 0.01
warmup_ratio = 0.1
logging_steps = 50
eval_steps = 200
save_steps = 200
gradient_accumulation_steps = 2

# LoRA config
lora_r = 8

lora_alpha = 32
lora_dropout = 0.1
target_modules = ["query", "key", "value", "dense"]  # works for many BERT-like models

# ============================================================
# 3. Load tokenized dataset from folder
# ============================================================
print("Loading tokenized dataset from:", tokenized_dataset_path)
dataset = load_from_disk(tokenized_dataset_path)

# Expecting a DatasetDict with splits: "train", "validation", "test"
# and columns: "input_ids", "attention_mask", "labels" (already tokenized)
print(dataset)

# ============================================================
# 4. Load tokenizer & FinBERT base model with 4-bit quantization (QLoRA)
# ============================================================
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)

print("Setting up 4-bit quantization config...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print("Loading base FinBERT model in 4-bit...")
model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# Prepare model for k-bit training (QLoRA)
model = prepare_model_for_kbit_training(model)

# Apply LoRA
print("Applying LoRA adapters...")
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="SEQ_CLS",
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# ============================================================
# 5. Data collator
# ============================================================
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ============================================================
# 6. Metrics (accuracy, f1 weighted/macro, precision, recall)
# ============================================================
def compute_metrics(eval_pred) -> Dict[str, float]:
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)

    metrics: Dict[str, Any] = {}
    metrics["accuracy"] = accuracy_score(labels, preds)

    metrics["f1_weighted"] = f1_score(labels, preds, average="weighted")
    metrics["f1_macro"] = f1_score(labels, preds, average="macro")

    metrics["precision_weighted"] = precision_score(labels, preds, average="weighted", zero_division=0)
    metrics["precision_macro"] = precision_score(labels, preds, average="macro", zero_division=0)

    metrics["recall_weighted"] = recall_score(labels, preds, average="weighted", zero_division=0)
    metrics["recall_macro"] = recall_score(labels, preds, average="macro", zero_division=0)

    return metrics

# ============================================================
# 7. TrainingArguments
# ============================================================
training_args = TrainingArguments(
    output_dir=final_model_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    warmup_ratio=warmup_ratio,
    logging_steps=logging_steps,
    gradient_accumulation_steps=gradient_accumulation_steps,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

# ============================================================
# 8. Trainer
# ============================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ============================================================
# 9. Train
# ============================================================
print("Starting training...")
train_result = trainer.train()
trainer.save_state()

print("Training metrics:")
for k, v in train_result.metrics.items():
    print(f"{k}: {v}")

# ============================================================
# 10. Evaluation: validation and test
# ============================================================

val_metrics  = trainer.evaluate(eval_dataset=dataset["validation"])
test_metrics = trainer.evaluate(eval_dataset=dataset["test"])

print("\nEvaluating on validation set...")
val_metrics = trainer.evaluate(eval_dataset=dataset["validation"])
print("Validation metrics:")
for k, v in val_metrics.items():
    print(f"{k}: {v}")

print("\nEvaluating on test set...")
test_metrics = trainer.evaluate(eval_dataset=dataset["test"])
print("Test metrics:")
for k, v in test_metrics.items():
    print(f"{k}: {v}")

# ============================================================
# 11. Save final model & tokenizer to Google Drive
# ============================================================
print("\nSaving final model and tokenizer to:", final_model_dir)
os.makedirs(final_model_dir, exist_ok=True)
trainer.save_model(final_model_dir)          # saves model + PEFT adapters
tokenizer.save_pretrained(final_model_dir)

print("Done.")

Loading tokenized dataset from: /content/drive/MyDrive/Unzip_Tokenised/tokenized_datasets
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 18391
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2299
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2299
    })
})
Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

Step,Training Loss
50,1.978300
100,1.300300
150,0.885000
200,0.815000
250,0.680700
300,0.629500
350,0.607600
400,0.571100
450,0.554400
500,0.519600


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Training metrics:
train_runtime: 3089.6433
train_samples_per_second: 17.857
train_steps_per_second: 1.117
total_flos: 1.4744163113330688e+16
train_loss: 0.4184676839303279
epoch: 3.0



Evaluating on validation set...
Validation metrics:
eval_loss: 0.3745986819267273
eval_accuracy: 0.8599391039582427
eval_f1_weighted: 0.8608359421092294
eval_f1_macro: 0.8370903729931732
eval_precision_weighted: 0.8652744149376367
eval_precision_macro: 0.8295530986034126
eval_recall_weighted: 0.8599391039582427
eval_recall_macro: 0.8505233701520384
eval_runtime: 35.477
eval_samples_per_second: 64.803
eval_steps_per_second: 4.059
epoch: 3.0

Evaluating on test set...
Test metrics:
eval_loss: 0.38792601227760315
eval_accuracy: 0.8616789908655937
eval_f1_weighted: 0.861919262076599
eval_f1_macro: 0.843522303340786
eval_precision_weighted: 0.8648360095485855
eval_precision_macro: 0.83811586934051
eval_recall_weighted: 0.8616789908655937
eval_recall_macro: 0.8528820356094707
eval_runtime: 35.52
eval_samples_per_second: 64.724
eval_steps_per_second: 4.054
epoch: 3.0

Saving final model and tokenizer to: /content/drive/MyDrive/finbert_qlora_finetuned
Done.
